In [26]:
from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator
from huggingface_hub import notebook_login
import psycopg2 as pg
from tqdm import tqdm

In [2]:
notebook_login()

In [3]:
dataset = load_dataset("dilovancelik/danish_law_qa")
dataset = dataset["train"].train_test_split(test_size=0.05)
dataset["validation"] = dataset["test"].remove_columns("context_id")
new_train_test = dataset["train"].train_test_split(test_size=0.05)
dataset["train"] = new_train_test["train"].remove_columns("context_id")
dataset["test"] = new_train_test["test"].remove_columns("context_id")


In [4]:
model_name = "sentence-transformers/all-distilroberta-v1"
model = SentenceTransformer(model_name)


In [5]:

evaluator_valid = TripletEvaluator(
    anchors=dataset["validation"]["question"],
    positives=dataset["validation"]["context"],
    negatives=dataset["validation"]["neg_context"],
    name="danish_law-validator",
    show_progress_bar=True
)
evaluator_valid(model)

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

{'danish_law-validator_cosine_accuracy': 0.8258547186851501}

In [6]:
loss = MultipleNegativesRankingLoss(model)

In [7]:
num_epochs = 1
batch_size = 16
lr = 2e-5
finetuned_model_name = "all-distilroberta-v1_danish_law_fine_tune"

train_args = SentenceTransformerTrainingArguments(
    output_dir=f"models/{finetuned_model_name}",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=lr,
    warmup_ratio=0.1,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=100,
)

In [8]:

trainer = SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    loss=loss,
    evaluator=evaluator_valid,
)
trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Danish Law-validator Cosine Accuracy
100,1.522000,1.010324,0.949786
200,1.099100,0.858892,0.957265
300,0.975900,0.804419,0.965812
400,0.850000,0.761589,0.965812
500,0.836900,0.736278,0.972222
600,0.828400,0.693604,0.971154
700,0.874700,0.679393,0.970085
800,0.761500,0.673524,0.970085
900,0.786200,0.665091,0.973291
1000,0.749000,0.663845,0.972222


Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

TrainOutput(global_step=1056, training_loss=0.9202810851010409, metrics={'train_runtime': 2154.5584, 'train_samples_per_second': 7.839, 'train_steps_per_second': 0.49, 'total_flos': 0.0, 'train_loss': 0.9202810851010409, 'epoch': 1.0})

In [10]:
evaluator_test = TripletEvaluator(
    anchors=dataset["test"]["question"],
    positives=dataset["test"]["context"],
    negatives=dataset["test"]["neg_context"],
    name="danish_law-test",
)
print("Validation:", evaluator_valid(model))
print("Test:", evaluator_test(model))

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Batches:   0%|          | 0/59 [00:00<?, ?it/s]

Validation: {'danish_law-validator_cosine_accuracy': 0.9732906222343445}
Test: {'ai-job-test_cosine_accuracy': 0.9718785285949707}


In [11]:
model.push_to_hub(f"dilovancelik/{finetuned_model_name}")

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

'https://huggingface.co/dilovancelik/all-distilroberta-v1_danish_law_fine_tune/commit/3518809092b2ce03fa813ad0eb356356ccb2de02'

In [56]:
def embed_documents(document, embedding):
    conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute(
        "INSERT INTO law_embeddings (context, embedding) VALUES (%s, %s)",
        (document, embedding),
    )
    cur.close()
    conn.close()


In [41]:
with open("documents.csv", "r") as f:
    documents = f.readlines()

In [57]:

with tqdm(total = len(documents), desc="generating questions") as pbar:
    for document in documents:
        embedding = model.encode(document).tolist()
        embed_documents(document, str(embedding))
        pbar.update(1)

generating questions: 100%|█████████████████████████████████████████████████████████████████████████| 3216/3216 [09:13<00:00,  5.81it/s]


In [58]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "Hvad siger loven om hvor man må bygge hus"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <=> %s::vector LIMIT 10;",
    (str(emb),),
)

result = cur.fetchall()
print(result)

cur.close()
conn.close()


finished embedding
[('Lov om ombygning af Aarhus H (intro): Lov om ombygning af Aarhus H VI MARGRETHE DEN ANDEN, af Guds Nåde Danmarks Dronning, gør vitterligt: Folketinget har vedtaget og Vi ved Vort samtykke stadfæstet følgende lov:\n',), ('Lov om ombygning af Aarhus H (Kapitel 2):  Miljømæssige vurderinger     § 2. Anlægsprojektet nævnt i § 1 skal gennemføres inden for rammerne af de udførte vurderinger af projektets indvirkninger på miljøet, jf. dog § 5, stk. 1.\n',), ('Lov om ombygning af Aarhus H (Kapitel 8):  Ikrafttræden     § 14. Loven træder i kraft den 1. juli 2022.\n',), ('Lov om ombygning af Aarhus H (Kapitel 9):  Ændringer i anden lovgivning   § 15. I ligningsloven, jf. lovbekendtgørelse nr. 1735 af 17. august 2021, som ændret bl.a. ved § 3 i lov nr. 2397 af 14. december 2021 og senest ved § 2 i lov nr. 2613 af 28. december 2021, foretages følgende ændring:  1. I § 7 indsættes som nr. 38:  »38) Kompensation og beløb ydet for gener og ulemper eller i forbindelse med genhus

In [70]:
from llama_index.llms.ollama import Ollama

In [88]:

PROMPT_TEMPLATE = """\
Kontekst er nedenfor.

---------------------
{context_str}
---------------------

Givet den givne kontekst vælg de 5 mest relevante love og besvar nedenstående forespørgsel:

Du er en advokat / jurist. Du er blevet spurgt følgende. Fortæl gerne hvis en lov er relevant, men ikke hvis den ikke er: \

{question}  \

"""


In [79]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "kan der nedlægges opholdsforbud i danmark"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <=> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n".join([r[0] for r in result])

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)

finished embedding
Ja, det kan der. Det er tilladt at nedlægge et opholdsforbud i Danmark i henhold til Lov om tilhold (Lov nr. 315 af 1958).


In [68]:
PROMPT_TEMPLATE.format(context_str="Hallo", question="Hej")

'Kontekst er nedenfor.\n\n---------------------\nHallo\n---------------------\n\nGivet den givne kontekst og ingen anden viden.\nSvar på nedenstående forespørgsel.\n\nDu er en advokat / jurist. Du er blevet spurgt følgende: \nHej  '

In [71]:
llm = Ollama(model="llama3")

In [80]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "kan der nedlægges opholdsforbud i danmark"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <=> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n".join([r[0] for r in result])

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)

finished embedding
Ja, det kan der. Ifølge lov om tilhold ( Lov nr. 335 af 7. juni 1999), kan ministeren for offentlige arbejder bemyndiges til at undlade at foretage opkrævning af de i medfør af § 4 i loven om ordning af afløbsforholdene ved Ringkøbing Fjord m.m. omhandlede lodsejerbidrag, som endnu ikke er indbetalt til statskassen (§ 1 i Lov nr. 141 af 30. marts 1942).


In [90]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "Må man have en pistol i Danmark"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <=> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n\n\n".join([r[0] for r in result])

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)

finished embedding
Efter at have læst de givne love og konteksten, kan jeg konkludere, at der ikke er nogen relevante love, der handler om anvendelse af pistoler i Danmark. Derimod kan jeg fortælle dig, at Lov om Danmarks tiltrædelse af De europæiske Fællesskaber (Tiltrædelsesloven) og Lov om Danpilot (Kapitel 8) ikke er relevante for dit spørgsmål.

Derimod kan jeg give dig en generel oplysning, nemlig at Danmark har strenge lover om våben i almindelighed. I Danmark kræves et tilladelse fra politiet og andre myndigheder for at indehabe eller anvende våben, herunder pistoler. Derudover er der begrænsninger for, hvem der kan indehave eller anvende våben, og der er strenge regler om sikkerhed og lagring af våben.

Hvis du har yderligere spørgsmål eller ønsker at vide mere om anvendelse af pistoler i Danmark, vil jeg være glad for at hjælpe.


In [94]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "Hvordan er loven i forhold til organiseret kriminalitet"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <=> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n\n\n".join([r[0] for r in result])
kilder = [r[0].split(":")[0] for r in result]

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)
print(kilder)

finished embedding
Basert på den givne kontekst, har jeg valgt de 5 mest relevante love og besvaret spørsmålet. Her er mine valg:

1. Lov om bekæmpelse af ungdomskriminalitet: Denne lov er relevant fordi den omhandler bekæmpelse af organiseret kriminalitet blandt unges.
2. Lov om Institut for Menneskerettigheder – Danmarks Nationale Menneskerettighedsinstitution1): Denne lov er relevant fordi den omhandler institutionens rolle i beskyttelse og fremme af menneskerettigheder, herunder retten til ligebehandling og frihed fra diskrimination.
3. Lov om Danmarks tiltrædelse af De europæiske Fællesskaber (Tiltrædelsesloven): Denne lov er relevant fordi den omhandler Danmarks medlemskab i Europæiske Fællesskaber, hvilket kan påvirke bekæmpelsen af organiseret kriminalitet.
4. Lov om ungdomskontakt: Denne lov er ikke relevant, da den ikke handler om organiseret kriminalitet.
5. Lov om lommepengejob: Denne lov er ikke relevant, da den ikke handler om organiseret kriminalitet.

I forhold til orga

In [ ]:
model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'
model = SentenceTransformer(model_name)

In [109]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "kan politiet forhindre mig i at være på en ejendom"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <-> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n\n\n".join([r[0] for r in result])
kilder = [r[0].split(":") for r in result]

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)
for kilde in kilder:
    print(f"{kilde[0]}:\n", )


finished embedding
Basert på den givne kontekst vil jeg identificere de 5 mest relevante love og besvare forespørgslen.

Relevante lover:

1. Lov om leje (Ejendomsbegrebet)
2. Ejendomsskatteloven
3. Lov om tinglysning
4. Lov om forfald
5. Lov om leje (Tinglysning)

Forespørgslen: Kan politiet forhindre mig i at være på en ejendom?

Svaret er NEJ, basert på de relevante lover.

Lov om leje (Ejendomsbegrebet) siger ikke noget om politiets mulighed til å forhindre noen i å være på en ejendom. Ejendomsskatteloven og lov om tinglysning handler om andre emner, mens lov om forfald bare beskriver reglerne for forfaldte lån.

Lov om leje (Tinglysning) fastsætter regler for tinglysning af dokumenter om overdragelse af en ejendom, hvorpå der er tilbudspligt. Dette lover ikke noget om politiets mulighed til å forhindre noen i å være på en ejendom.

Derfor kan politiet ikke forhindre dig i at du er på en ejendom, basert på de relevante lover.
Lov om leje (Ejendomsbegreber):

Ejendomsskattelov (Raba

In [109]:
#query = input("Hvilken lov vil du gerne søge efter\n")
query = "kan politiet forhindre mig i at være på en ejendom"
emb = model.encode(query).tolist()
print("finished embedding")

conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT context FROM law_embeddings ORDER BY embedding <-> %s::vector LIMIT 20;",
    (str(emb),),
)

result = cur.fetchall()

contekst = "\n\n\n".join([r[0] for r in result])
kilder = [r[0].split(":") for r in result]

prompt = PROMPT_TEMPLATE.format(context_str=contekst, question=query)

cur.close()
conn.close()

res = llm.complete(prompt)
print(res)
for kilde in kilder:
    print(f"{kilde[0]}:\n", )


finished embedding
Basert på den givne kontekst vil jeg identificere de 5 mest relevante love og besvare forespørgslen.

Relevante lover:

1. Lov om leje (Ejendomsbegrebet)
2. Ejendomsskatteloven
3. Lov om tinglysning
4. Lov om forfald
5. Lov om leje (Tinglysning)

Forespørgslen: Kan politiet forhindre mig i at være på en ejendom?

Svaret er NEJ, basert på de relevante lover.

Lov om leje (Ejendomsbegrebet) siger ikke noget om politiets mulighed til å forhindre noen i å være på en ejendom. Ejendomsskatteloven og lov om tinglysning handler om andre emner, mens lov om forfald bare beskriver reglerne for forfaldte lån.

Lov om leje (Tinglysning) fastsætter regler for tinglysning af dokumenter om overdragelse af en ejendom, hvorpå der er tilbudspligt. Dette lover ikke noget om politiets mulighed til å forhindre noen i å være på en ejendom.

Derfor kan politiet ikke forhindre dig i at du er på en ejendom, basert på de relevante lover.
Lov om leje (Ejendomsbegreber):

Ejendomsskattelov (Raba

In [162]:
METTE_PROMPT_TEMPLATE = """\
Du er en LLM som giver svarer på hvad Mette Frederiksen syntes om {question}. \
Du må kun besvarer baseret af de nedenstående citater, Du skal kun komme med et enkelt svar som er på Dansk \

---------------------
{context_str}
---------------------
"""


In [177]:

query = input("Hvilket emne vil du høre Mette Frederiksens mening om?\n\n")
print("Tænker ...")
emb = model.encode(query).tolist()

print("Henter citater...")
conn = pg.connect("dbname=vector_rag user=postgres password=postgres")
conn.autocommit = True
cur = conn.cursor()

cur.execute(
    "SELECT speech, context, line FROM speeches_embeddings_v2 ORDER BY embedding <-> %s::vector LIMIT 5;",
    (str(emb),),
)

result = cur.fetchall()

context_all = []
speeches =  []
for row in result:
    speech = row[0]
    speeches.append(speech)
    context_all.append(f"'{row[1]}'")
    line = row[2]

prompt = METTE_PROMPT_TEMPLATE.format(context_str="\n\n".join(list(set(context_all))), question=query)

cur.close()
conn.close()

print("Prøver at formulere mig...")
res = llm.complete(prompt)
print(res)
print("\n\nLink til Taler og Citater som er valgt:")
for row in result:
    print(f"Citat: '{row[2].strip()}'\t\nLink: https://www.dansketaler.dk/tale/{row[0]}\n")

Hvilket emne vil du høre Mette Frederiksens mening om?

 Corona nedlukning


Tænker ...
Henter citater...
Prøver at formulere mig...
Her er Mette Frederiksens syn på Corona nedlukning:

"Jeg er overbevist om, at nedlukningen af dele af vores samfund var rigtig. At være forsigtig, når vi stod med en ukendt, uforudsigelig og dødelig virus – det var rigtigt."

"Hviskort har vi måttet sætte hensynet til hinanden først. Hver især. Nogle synes, at den pris er høj. Det forstår jeg godt. Men alternativet var værre. Det er det stadigvæk."


Link til Taler og Citater som er valgt:
Citat: 'Situationen er alvorlig. Derfor har vi forlænget nedlukningen af store dele af samfundet indtil midt i januar.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-nytaarstale-2021

Citat: 'Det er nu en uge siden, at vi lukkede Danmark delvist ned.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-ved-pressemoede-om-anden-coronanedlukning

Citat: 'Sådan har det også været under corona.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-ved-folketingets-a

In [111]:
model_name = 'Snowflake/snowflake-arctic-embed-l-v2.0'
model = SentenceTransformer(model_name)

In [158]:
for con in context_all:
    lines = con.splitlines()
    
    print("lines: ",len(lines), " characters: ", len(con))

lines:  40  characters:  2136
lines:  20  characters:  1670
lines:  20  characters:  737
lines:  92  characters:  3799
lines:  20  characters:  1620


In [129]:
lines = context_all[0].splitlines()

In [151]:
lines = []
char_split = False
for line in raw_lines:
    if line != "\n":
        lines.append(line)
    if re.search("[a-zA-Z]", line) is None and "\n" != line:
        char_split = True
context_splits = []

if char_split:
    context = []
    for line in lines:
        if re.search("[a-zA-Z]", line) is None and "\n" != line:
            context_splits.append(context)
            context = []
        else:
            context.append(line)
    if context != []:
        context_splits.append(context)
else:
    chunk_size = 10  # group size
    overlap = 2  # overlap size
    context_splits = [
        lines[i : i + chunk_size]
        for i in range(0, len(lines), chunk_size - overlap)
    ]

In [152]:
context_splits

[['Tale\n',
  'Tak for invitationen. Jeg har set frem til at være med i dag.\n',
  'Politik er fyldt med oneliners.\n',
  '“Never give in,” sagde Winston Churchill.\n',
  '“It’s morning again in America,” lød det fra Ronald Reagan.\n',
  'Barack Obama sagde: “Yes we can.”\n',
  'Og så er der Angela Merkels “Wir schaffen das”.\n',
  'Enig eller uenig. Når politiske slogans er bedst, indfanger de kernen af, hvad det hele handler om.\n',
  'I fredags hørte jeg en borgmester, der med tre ord fangede mit syn på danske kommuner. \xa0\n',
  'Han sagde: ”Vi fikser det.”\n',
  'Det er præcis det, I gør – alle jer, der er til stede i dag.\n',
  'Borgmestre. Kommunalbestyrelsesmedlemmer. Embedsmænd. Og mange andre lokale ildsjæle.\xa0 \xa0\n',
  'I fikser velfærdssamfundet i dagligdagen.\n',
  'Det er jer, der er tættest på danskerne.\n',
  'Jeres medarbejdere, der hver morgen står klar til at modtage den næste generation i vuggestuen og børnehaven.\n',
  'Jeres medarbejdere, der klæder skolebørn

Citat: 'Og når vi siger, vi vil styrke psykiatrien.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-til-socialdemokratiets-arsmode

Citat: 'Og jeg glæder mig til at høre jeres.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-ved-kommunalpolitisk-topmode

Citat: 'Vi har i dag en særlig udfordring i vores behandling af mennesker med psykiske lidelser. Som ofte er de mest sårbare af os alle.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-ved-folketingets-genaabning-2023

Citat: 'Og så skal vi forbedre psykiatrien.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-aarsmoedetale-2022

Citat: 'Nu vil jeg gerne tale om tryghed.'	
Link: https://www.dansketaler.dk/tale/mette-frederiksens-tale-ved-socialdemokratiets-kongres-2017



In [150]:
with open("taler/mette-frederiksens-tale-ved-kommunalpolitisk-topmode.txt", "r") as f:
    raw_lines = f.readlines()

In [169]:
result

[('mette-frederiksens-tale-til-socialdemokratiets-arsmode',
  'Kære venner,\xa0\n\nJeg skal til at slutte min tale.\xa0\n\nVi vil bruge de kommende år til at gøre velfærdssamfundet stærkere.\xa0\n\nVi skal i mål med lønløftet for at sikre flere medarbejdere.\xa0\n\nMen der er også andre vigtige opgaver foran os.\xa0\n\nVi skal fortsætte vores store engagement i Ukraine. Det er fremtidens Europa, der formes i de her år.\xa0\n\nAlt det vi tror på – alt det vi står for - vil tabe, hvis det igen er den stærke mand og magt, der definerer andre landes grænser.\xa0\n\nRuslands krig er en gammeldags og ondskabsfuld krig. Med tusinder og atter tusinder ofre.\xa0\n\nJeg bliver ofte spurgt hvad vi gør, hvis europæerne bliver trætte af krigen. Helt ærligt. Så længe ukrainerne kæmper, så kan ingen af os tillade os at vige fra deres side.\xa0\n\nDer er få øjeblikke i verdenshistorien, der definerer fremtiden og menneskeheden så meget som netop nu. Det er ikke Putin. Det er Zelensky, der skal vinde d